In [ ]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import os, time
from tqdm import tqdm

In [ ]:
# Login to your W&B account
wandb.login()

# Initialize a new W&B project
wandb.init(project="tiny-imagenet-resnet", config={
    "epochs": 100,
    "batch_size": 256,
    "learning_rate": 0.001,
    "model": "ResNet18",
    "optimizer": "Adam",
    "img_size": 128
})

config = wandb.config


In [ ]:
import shutil

data_dir = "/home/shiva/Assignment-6/wandb/ass_6/tiny-imagenet-200"
train_dir = os.path.join(data_dir, "train")
val_dir   = os.path.join(data_dir, "val")
val_img_dir = os.path.join(val_dir, "images")
val_annot_file = os.path.join(val_dir, "val_annotations.txt")

if os.path.exists(val_img_dir) and os.path.exists(val_annot_file):
    print("Organizing validation folder...")
    with open(val_annot_file, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                img, cls = parts[0], parts[1]
                cls_folder = os.path.join(val_dir, cls)
                os.makedirs(cls_folder, exist_ok=True)
                src = os.path.join(val_img_dir, img)
                dst = os.path.join(cls_folder, img)
                if os.path.exists(src) and not os.path.exists(dst):
                    shutil.copy(src, dst)
    print("✅ Validation folder organized successfully.")
else:
    print("Validation folder already organized.")


Validation folder already organized.


In [ ]:
import shutil, os

# Path to your validation directory
val_dir = "/home/shiva/Assignment-6/wandb/ass_6/tiny-imagenet-200/val"
# Remove leftover 'images' folder if it exists
bad_folder = os.path.join(val_dir, "images")
if os.path.exists(bad_folder):
    shutil.rmtree(bad_folder)
    print("✅ Removed extra folder:", bad_folder)
else:
    print("No extra folder found.")

# Reload validation dataset
from torchvision import datasets

val_data = datasets.ImageFolder(val_dir, transform=transform_val)
val_loader = DataLoader(val_data, batch_size=config.batch_size, shuffle=False, num_workers=2, pin_memory=True)

print(f"✅ Validation dataset reloaded. Classes: {len(val_data.classes)}")


No extra folder found.
✅ Validation dataset reloaded. Classes: 200


In [ ]:
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(config.img_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

transform_val = transforms.Compose([
    transforms.Resize(config.img_size),
    transforms.CenterCrop(config.img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

train_data = datasets.ImageFolder(train_dir, transform=transform_train)
val_data   = datasets.ImageFolder(val_dir,   transform=transform_val)

train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_data,   batch_size=config.batch_size, shuffle=False, num_workers=2, pin_memory=True)

print(f"✅ Train samples: {len(train_data)} | Val samples: {len(val_data)} | Classes: {len(train_data.classes)}")


✅ Train samples: 100000 | Val samples: 10000 | Classes: 200


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = models.resnet18(pretrained=True)

# Freeze backbone
for param in model.parameters():
    param.requires_grad = False

# Replace final FC layer for 200 Tiny ImageNet classes
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 200)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=config.learning_rate)

print("✅ Model ready for training")


In [ ]:
torch.backends.cudnn.benchmark = True
best_val_acc = 0.0
best_path = "/home/shiva/Assignment-6/wandb/ass_6/model/best_resnet_tiny_imagenet.pth"

for epoch in range(config.epochs):
    model.train()
    running_loss, correct, total = 0, 0, 0

    # TRAIN LOOP
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs} - Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    train_loss = running_loss / len(train_loader)

    # VALIDATION LOOP (optimized to prevent crash)
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{config.epochs} - Validating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            val_total += labels.size(0)
            val_correct += preds.eq(labels).sum().item()

            # clear memory after each batch
            del images, labels, outputs, preds, loss
            torch.cuda.empty_cache()

    val_loss = val_loss / val_total
    val_acc  = 100.0 * val_correct / val_total

    # Log metrics to W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "train_acc": train_acc,
        "val_acc": val_acc
    })

    print(f"Epoch {epoch+1}/{config.epochs} -> TrainAcc: {train_acc:.2f}% | ValAcc: {val_acc:.2f}%")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_path)
        wandb.save(best_path)
        print(f"✅ Best model updated (Val Acc: {val_acc:.2f}%)")

print(f"\nTraining Complete ✅ | Best Validation Accuracy: {best_val_acc:.2f}%")


Epoch 1/100 - Training: 100%|██████████| 391/391 [00:35<00:00, 10.92it/s]


Epoch 1/100 -> TrainAcc: 31.91% | ValAcc: 50.77%


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


✅ Best model updated (Val Acc: 50.77%)


Epoch 2/100 - Training: 100%|██████████| 391/391 [00:33<00:00, 11.64it/s]


...

Epoch 98/100 -> TrainAcc: 45.55% | ValAcc: 54.78%


Epoch 99/100 - Training: 100%|██████████| 391/391 [00:32<00:00, 11.90it/s]


Epoch 99/100 -> TrainAcc: 45.80% | ValAcc: 55.17%


Epoch 100/100 - Training: 100%|██████████| 391/391 [00:33<00:00, 11.66it/s]
                                                                           

Epoch 100/100 -> TrainAcc: 45.87% | ValAcc: 54.99%

Training Complete ✅ | Best Validation Accuracy: 55.58%


In [ ]:
artifact = wandb.Artifact('resnet_tiny_imagenet_model', type='model')
artifact.add_file(best_path)
wandb.log_artifact(artifact)

print("✅ Model artifact logged to W&B")


✅ Model artifact logged to W&B


In [ ]:
!pip install huggingface_hub --quiet

In [ ]:

from huggingface_hub import notebook_login

notebook_login()  # ⬅️ This will ask for your Hugging Face token


In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository

repo_name = "resnet18-tinyimagenet"
user = HfApi().whoami()["name"]
repo_id = f"{user}/{repo_name}"

# Create repo on your HF account (skip if it already exists)
HfApi().create_repo(repo_id=repo_id, private=False, exist_ok=True)
print(f"✅ Repository created: https://huggingface.co/{repo_id}")


In [ ]:
import torch, os

repo_name = "resnet18-tinyimagenet"
user = "Shiva-12345"
repo_id = f"{user}/{repo_name}"

model_save_dir = f"/home/shiva/Assignment-6/wandb/ass_6/{repo_name}"
os.makedirs(model_save_dir, exist_ok=True)

# ✅ Save model weights
torch.save(model.state_dict(), f"{model_save_dir}/pytorch_model.bin")



In [ ]:
import os
os.environ["HF_USER"] = "enter your user name"
os.environ["SPACE_NAME"] = "resnet18-tinyimagenet-app"
os.environ["HF_TOKEN"] = "enter your hf token"

In [ ]:
%%writefile app.py
import gradio as gr
import torch
from torchvision import models, transforms
from PIL import Image

# Load label names (Tiny ImageNet classes)
label_file = "wnids.txt"  # optional file with class IDs if you have it
if not torch.cuda.is_available():
    print("Running on CPU")

# Build label list (1-200 if wnids.txt not available)
if not hasattr(__builtins__, 'open'):
    open = __builtins__.open
try:
    with open(label_file) as f:
        idx_to_class = [line.strip() for line in f.readlines()]
except FileNotFoundError:
    idx_to_class = [f"class_{i}" for i in range(200)]

# Load model
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 200)
model.load_state_dict(torch.load("pytorch_model.bin", map_location="cpu"))
model.eval()

# Preprocess
transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# Prediction function
def predict(img):
    img = transform(img).unsqueeze(0)
    with torch.no_grad():
        outputs = model(img)
        pred = torch.argmax(outputs, dim=1).item()
    label = idx_to_class[pred] if pred < len(idx_to_class) else str(pred)
    return f" Predicted: {label}"

# Build interface
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload Tiny ImageNet Image"),
    outputs=gr.Textbox(label="Prediction"),
    title=" ResNet18 Tiny ImageNet Classifier",
    description="Upload an image and get the predicted class name."
)

if __name__ == "__main__":
    demo.launch()


Overwriting app.py


In [ ]:
%%writefile requirements.txt
torch
torchvision
gradio


Overwriting requirements.txt


In [ ]:
import os, shutil, tempfile
from huggingface_hub import HfApi

token = os.environ.get("HF_TOKEN")
user = os.environ.get("HF_USER")
space = os.environ.get("SPACE_NAME")
if not token or not user or not space:
    raise RuntimeError("HF_TOKEN, HF_USER or SPACE_NAME not set in environment.")

api = HfApi(token=token)
repo_id = f"{user}/{space}"

repo_url = api.create_repo(repo_id=repo_id, repo_type="space", space_sdk="gradio", exist_ok=True)
print("Using Space:", repo_url)

tmpdir = tempfile.mkdtemp(prefix="hf_space_")
print("Preparing upload folder:", tmpdir)

# copy notebook artifacts into tmpdir
cwd = os.getcwd()
copied = []
for fname in ("app.py", "requirements.txt"):
    src = os.path.join(cwd, fname)
    if os.path.exists(src):
        shutil.copy(src, tmpdir)
        copied.append(src)
    else:
        print(f"Warning: {src} not found, skipping.")

# copy model file from model_save_dir
model_bin = os.path.join(model_save_dir, "pytorch_model.bin")
if os.path.exists(model_bin):
    shutil.copy(model_bin, tmpdir)
    copied.append(model_bin)
else:
    print(f"Warning: {model_bin} not found, skipping.")

if not copied:
    raise RuntimeError("No files found to upload. Ensure app.py, requirements.txt or model exist.")

api.upload_folder(
    folder_path=tmpdir,
    repo_id=repo_id,
    repo_type="space",
    commit_message="🚀 Deploy ResNet18 Tiny ImageNet Gradio App"
)

print("Uploaded files:", copied)
print("✅ App deployed successfully at:", repo_url)

In [ ]:
import torch
from torchvision import transforms, datasets
import wandb
import numpy as np
from tqdm import tqdm

# Re-initialize W&B run for Q3
wandb.init(project="tiny-imagenet-drift-detection",
           name="baseline-run",
           config={"epochs": 1, "batch_size": 16, "model": "ResNet18"},
           reinit=True)


In [ ]:
# load your trained model weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device)

# normal (clean) validation loader already defined as val_loader
criterion = torch.nn.CrossEntropyLoss()

def evaluate_model(model, dataloader, desc="eval"):
    model.eval()
    correct, total, loss_val = 0, 0, 0.0
    with torch.no_grad():
        for imgs, labels in tqdm(dataloader, desc=desc):
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            loss = criterion(out, labels)
            loss_val += loss.item()
            _, preds = out.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()
    acc = 100 * correct / total
    return acc, loss_val / len(dataloader)

baseline_acc, baseline_loss = evaluate_model(model, val_loader, desc="Baseline Validation")

wandb.log({"baseline_acc": baseline_acc, "baseline_loss": baseline_loss})
print(f"✅ Baseline Accuracy: {baseline_acc:.2f}%")
wandb.finish()


In [ ]:
# Drifted transform (simulate brightness shift + noise)
drift_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ColorJitter(brightness=1.5, contrast=1.5),  # drift in brightness/contrast
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x + 0.05 * torch.randn_like(x)),  # add Gaussian noise
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

# New dataset with drift
val_data_drift = datasets.ImageFolder(val_dir, transform=drift_transform)
val_loader_drift = torch.utils.data.DataLoader(val_data_drift, batch_size=16, shuffle=False, num_workers=2)


In [ ]:
wandb.init(project="tiny-imagenet-drift-detection",
           name="drifted-run",
           config={"drift_type": "brightness+noise"},
           reinit=True)

drifted_acc, drifted_loss = evaluate_model(model, val_loader_drift, desc="Drifted Validation")

wandb.log({
    "drifted_acc": drifted_acc,
    "drifted_loss": drifted_loss,
    "baseline_acc": baseline_acc
})
print(f"⚠️ Drifted Accuracy: {drifted_acc:.2f}%")

# Alert if drop > threshold
threshold = baseline_acc * 0.8   # e.g., 20% drop allowed
if drifted_acc < threshold:
    wandb.alert(
        title="⚠️ Accuracy Drop Detected!",
        text=f"Drifted accuracy {drifted_acc:.2f}% below threshold {threshold:.2f}%",
        level=wandb.AlertLevel.WARN
    )
    print("🚨 W&B alert triggered!")
else:
    print("✅ Accuracy within acceptable range.")
wandb.finish()